In [1]:
import pandas as pd
import re
import random
import numpy as np

In [19]:
df = pd.read_csv('bicicletas/AccidentesBicicletas_2018.csv', encoding='latin_1',sep = ';')

In [35]:
df[df.columns[7:13]]

,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve
0,NO,NO,NO,NO,SI,NO
1,NO,NO,NO,NO,SI,NO
2,NO,NO,NO,NO,SI,NO
3,NO,NO,NO,NO,SI,NO
4,NO,NO,NO,NO,SI,NO
...,...,...,...,...,...,...
695,NO,NO,NO,NO,SI,NO
696,NO,NO,NO,NO,SI,NO
697,NO,NO,NO,NO,SI,NO
698,NO,NO,NO,NO,SI,NO


In [36]:
lst_old_col = list(df.columns[7:13])

In [33]:
dict_weather = {
    'CPFA Granizo':'Granizo',
    'CPFA Hielo': 'Hielo',
    'CPFA Lluvia': 'Lluvia',
    'CPFA Niebla': 'Niebla',
    'CPFA Seco': 'Despejado',
    'CPFA Nieve': 'Nieve',
}

['CPFA Granizo',
 'CPFA Hielo',
 'CPFA Lluvia',
 'CPFA Niebla',
 'CPFA Seco',
 'CPFA Nieve',
 'CPSV Mojada']

In [2]:
def get_df_norm_header_index(path):
    lst_header = ['num_exp', 'dia', 'hora', 'calle', 'num', 'distrito', 'tipo_accidente', 'meteo', 'vehiculo', 'persona', 'edad', 'sexo', 'lesividad' ]
    df = pd.read_csv(path, encoding='latin_1',sep = ';', names=lst_header)
    df.drop(0, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

bici = get_df_norm_header_index('bicicletas/AccidentesBicicletas_2019.csv')


In [4]:
def create_date(df):
    df['fecha'] = df.dia+' '+df.hora
    df['fecha'] = pd.to_datetime(df['fecha'], dayfirst=True)
    df['fecha'] = df['fecha'].dt.round('H')
    df.drop(['dia', 'hora'], axis=1, inplace = True)
    return df

bici = create_date(bici)

In [5]:
def create_dir(df):
    df.num.apply(lambda x: x.strip())
    df.loc[(df.num == '-'), 'num']=''
    df['direccion'] = df.calle+' '+df.num
    df.direccion.apply(lambda x: x.strip())
    df.drop(['calle', 'num'], axis=1, inplace = True)
    return df

bici = create_dir(bici)

In [8]:
def fill_drop(df):
    df.lesividad.fillna('14', inplace=True)
    df.meteo.fillna('Se desconoce', inplace=True)
    df.drop(['sexo', 'vehiculo', 'persona', 'edad', 'num_exp'], axis=1, inplace=True)
    return df

bici = fill_drop(bici)

In [9]:
def change_accident(values):
    dict_accidente = {
        r'Caída':'Caida',
        r'Colisión':'Colision',
        r'Alcance':'Colision',
        r'Atropello':'Imprudencia',
        r'Choque':'Imprudencia'}
    for k, v in dict_accidente.items():
        if re.match(k, values):
            return v
    return 'Otro'

bici.tipo_accidente = bici.tipo_accidente.apply(lambda x: change_accident(x))

In [12]:
def change_injury(values):
    dict_injury = {
        '01':'Moderada','02':'Moderada',
        '03':'Grave',
        '04':'Fallecido',
        '05':'Leve','06':'Leve','07':'Leve','14':'Leve'}
    return dict_injury[values]


bici.lesividad = bici.lesividad.apply(lambda x: change_injury(str(x)))

In [15]:
bici = bici[['fecha','meteo', 'tipo_accidente', 'lesividad', 'distrito', 'direccion']]

In [16]:
bici.head()

,fecha,meteo,tipo_accidente,lesividad,distrito,direccion
0,2019-01-01 14:00:00,Despejado,Colision,Moderada,SALAMANCA,CALL. CASTELLO / CALL. DON RAMON DE LA CRUZ
1,2019-01-02 21:00:00,Despejado,Colision,Grave,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALES
2,2019-01-03 14:00:00,Se desconoce,Colision,Leve,VILLA DE VALLECAS,CALL. FELIPE ALVAREZ 10
3,2019-01-03 13:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA
4,2019-01-03 13:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA


In [17]:
bici.meteo.value_counts()

Despejado         703
Se desconoce       30
Lluvia débil       27
Nublado            13
LLuvia intensa      5
Granizando          1
Name: meteo, dtype: int64

In [ ]:
# encoding='cp850'
# encoding='cp1250'

# **
# encoding='latin_1'
# encoding='iso8859_15'

In [18]:
abr = pd.read_csv('tiempo/Anio2019/abr_meteo19.csv', encoding='latin_1',sep = ';')

In [20]:
abr.head(20)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,102,81,28079102_81_98,2019,4,1,1.90,V,...,1.45,V,0.90,V,0.97,V,1.68,V,1.12,V
1,28,79,102,81,28079102_81_98,2019,4,2,0.40,V,...,2.48,V,2.05,V,1.88,V,0.95,V,1.10,V
2,28,79,102,81,28079102_81_98,2019,4,3,1.18,V,...,3.95,V,3.53,V,3.40,V,3.42,V,2.58,V
3,28,79,102,81,28079102_81_98,2019,4,4,2.35,V,...,3.12,V,2.52,V,2.30,V,2.13,V,1.88,V
4,28,79,102,81,28079102_81_98,2019,4,5,3.02,V,...,2.88,V,1.43,V,0.93,V,2.18,V,2.58,V
5,28,79,102,81,28079102_81_98,2019,4,6,2.18,V,...,4.93,V,2.98,V,3.07,V,1.95,V,1.83,V
6,28,79,102,81,28079102_81_98,2019,4,7,2.77,V,...,1.73,V,1.48,V,0.73,V,0.37,V,0.75,V
7,28,79,102,81,28079102_81_98,2019,4,8,1.18,V,...,3.95,V,2.82,V,2.50,V,4.33,V,1.30,V
8,28,79,102,81,28079102_81_98,2019,4,9,1.22,V,...,3.48,V,3.92,V,3.15,V,2.98,V,3.67,V
9,28,79,102,81,28079102_81_98,2019,4,10,1.85,V,...,2.02,V,1.65,V,1.68,V,1.73,V,1.60,V
